## Imports

In [1]:
import json
import datetime
import pandas as pd
import collections
from collections import Counter
from collections import defaultdict
import csv
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as la
import time
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from array import array
import math
import operator
import itertools
import igraph

In [2]:
df_tweets = pd.read_json('file.json', orient ='split', compression = 'infer') 
df_tweets = df_tweets.fillna(value=np.nan)

In [3]:
def getrts(df_tweets):
    retweets_ = df_tweets["retweeted_status"].apply(lambda x:0 if str(x)=='nan' else 1)
    return retweets_

## Link Analysis

Generate the retweet graph

In [4]:
#We recieve a list of 0 if tweet is unique and 1 if it is a retweet
retweets = getrts(df_tweets)
#We get the index of the retweets
retweets = retweets[retweets == 1]
retweets = retweets.index.values
#We now get df_unique which are the original tweets and df_retweeted which are the retweeted tweets
df_unique = df_tweets.drop(retweets,axis=0)
df_unique = df_unique.reset_index(drop=True)
df_retweeted = df_tweets.loc[retweets]
df_retweeted = df_retweeted[df_retweeted['retweeted_status'].notna()]
df_retweeted = df_retweeted.reset_index(drop=True)

In [5]:
#We get common columns between the retweeted and the original tweets 
cols = df_retweeted.columns&df_retweeted.at[0,'retweeted_status'].keys()

for i in range(0,len(df_retweeted)):
    for col in cols:
        if col in df_retweeted.at[i,'retweeted_status'].keys():
            #Substitute the columns of the retweeted tweet by the original one
            if col != 'possibly_sensitive':
                df_retweeted.at[i,col] = df_retweeted.at[i,'retweeted_status'][col]

In [6]:
df_tweets = pd.concat([df_unique, df_retweeted], ignore_index=True)
df_tweets = df_tweets.drop_duplicates(subset=['id'])
df_tweets = df_tweets.reset_index(drop=True)

In [7]:
edges1 = [None]*len(df_retweeted)
edges2 = [None]*len(df_retweeted)
for i in range(0,len(df_retweeted)):
    edges1[i] = df_retweeted.at[i,'user']['screen_name']
    edges2[i] = df_retweeted.at[i,'retweeted_status']['user']['screen_name']

In [8]:
edges = {'rt':edges1, 'og':edges2}
edges = pd.DataFrame(edges)

In [9]:
tuples = [tuple(x) for x in edges.values]
graph = igraph.Graph.TupleList(tuples, directed = True)

We extract the 20% of edges as test-set, calling it T

In [10]:
# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(graph.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N),replace=False)

In [11]:
T = set()
edges_to_delete = set()
for idx, one_edge in enumerate(graph.es):
    
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target
    edges_to_delete.add(one_edge)
    if idx in test_idxs:
        T.add((n1, n2,1))

We extract from the graph generated from the training-set, a number of edges at distance-2 corresponding to the same of the test-set. This list is called D2

In [12]:
ground_truth = set()
trainset = set()
for idx, one_edge in enumerate(graph.es):
    
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target
    
    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
    else:
        trainset.add((n1, n2))

In [13]:
def find_nodes_at_distance_2(graph):
    """
    starting from a graph this function returns all the nodes at distance 2
    """
    

    all_potential_recommendations = set()
    
    for n1 in graph.vs:

        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(vertices=n1, order=1))

        # all the nodes at distance 1 and distance 2
        nodes_at_most_distant_2 = set(graph.neighborhood(vertices=n1, order=2))


        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1


        # check if empty set
        if len(only_nodes_at_distance_2) > 0:


            for n2 in only_nodes_at_distance_2:

                # since n1 is an igraph vertex object, we need to extract the id
                n1_index = n1.index


                all_potential_recommendations.add((n1_index, n2,0))
            
            
    return all_potential_recommendations

In [14]:
D2_aux = find_nodes_at_distance_2(graph)

In [15]:
count = 0
D2 = set()
for i in D2_aux:
    if count<=len(T):
        D2.add(i)
    count+=1

We merge the list of T and D2 in a unique list called U and then predict for each algorithm the existence (1) or not (0) of each edge in U. Half of them will have label 1 (T) and half of them will have label (0).
This link-prediction task is similar to the classic binary classification task.

In [18]:
U = D2.union(T)
d = {'Source':[i[0] for i in U],'Target':[i[1] for i in U],'Correct':[i[2] for i in U]}
df_test = pd.DataFrame(d)
df_test.head()

,Source,Target,Correct
0,24968,24968,1
1,7953,7953,1
2,29207,29207,1
3,9641,9641,1
4,25183,25183,1


ADA

In [19]:
def compute_ADA(u,v, graph):
    """
    compute adamic-adar from scratch
    """
    
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    
    # set Z of neighbors of both
    bridges = set(outlinks_from_u).intersection(inlinks_to_v)

    # degree of nodes in set Z
    deg_ = [graph.degree(n) for n in bridges]
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

In [20]:
df_test['ADA_score'] = 0
for i in range(0,len(df_test)):
    df_test.at[i,'ADA_score'] = compute_ADA(df_test.at[i,'Source'],df_test.at[i,'Target'],graph)

df_test["ADA_score"] = df_test["ADA_score"].apply(lambda x: round(x))
df_test["ADA_score"] = df_test["ADA_score"].apply(lambda x:1 if x == 2 else x)

In [21]:
right_predictions = len(df_test[df_test.Correct == df_test.ADA_score])

# accuracy
right_predictions/len(df_test)

0.0

In [22]:
df_test = df_test.drop('ADA_score',axis=1)

Jaccard

In [23]:
def compute_Jaccard(u,v, graph):
    """
    compute jaccard similarity
    """
    
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    # intesection of the two sets
    num = set(outlinks_from_u).intersection(inlinks_to_v)
    
    # union of the two sets
    den = set(outlinks_from_u).union(inlinks_to_v)
    
    # final division
    out = len(num)/len(den)
    
    return out

In [24]:
df_test['Jaccard_score'] = 0
for i in range(0,len(df_test)):
    df_test.at[i,'Jaccard_score'] = compute_Jaccard(df_test.at[i,'Source'],df_test.at[i,'Target'],graph)

df_test["Jaccard_score"] = df_test["Jaccard_score"].apply(lambda x: round(x))
df_test["Jaccard_score"] = df_test["Jaccard_score"].apply(lambda x:1 if x == 2 else x)

In [25]:
right_predictions = len(df_test[df_test.Correct == df_test.Jaccard_score])

# accuracy
right_predictions/len(df_test)

1.0

Personalized Pagerank. Personalized pagerank is suggested to be used for recommending a list of top-k new edges for each node.

In [26]:
def print_top_k(graph, topk, vid=None):
        
    # here we need also the argument vid, which corresponds to a node-id
    pr = enumerate(graph.personalized_pagerank(reset_vertices=vid))

    out = sorted(pr, key=lambda x: x[1], reverse=True)[:topk]
        
    return out

In [27]:
for i in df_test['Source'].unique():
    aux = print_top_k(graph, 11, i)
    aux = [j[0] for j in aux]
    aux.remove(i)
    print('For node',i,'edge connecting with the top-10 following nodes are recommended:',aux)

For node 24968 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 7953 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 29207 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 9641 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 25183 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 4191 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 14 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 11732 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 6129 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 28862 ed

For node 6148 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 8899 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 16502 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 12128 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 25784 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 24358 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 12640 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 4854 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 20046 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 2508

For node 23328 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 11715 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 17878 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 17830 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 29395 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 11365 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 250 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 9939 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 13303 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 2655

For node 26809 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 11267 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 5817 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 26459 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 26411 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 17597 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 20961 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 20611 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 79 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 6146 

For node 21672 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 18260 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 25648 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 20198 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 11695 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 8283 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 4106 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 23475 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 2833 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 1446

...

For node 19402 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 3511 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 3463 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 13602 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 213 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 15405 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 22946 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 3840 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 9955 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 5931 ed

For node 4174 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 22419 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 1427 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 8815 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 3365 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 5977 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 12442 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 4503 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 14795 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 18159 e

For node 9364 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 3761 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 28532 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 5852 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 15541 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 2335 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 21656 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 29197 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 17282 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 5564 

For node 10937 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 448 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 1060 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 3151 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 10539 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 16204 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 14778 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 26343 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 28434 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 16519

For node 24854 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 11600 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 17265 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 6150 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
For node 15489 edge connecting with the top-10 following nodes is recommended: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


ALS

In [31]:
from scipy.sparse import csr_matrix
import implicit

In [32]:
# first we get the adjacency matrix data
M = graph.get_adjacency().data
M = csr_matrix(M)

In [33]:
# here we run the model ALS
model = implicit.als.AlternatingLeastSquares(factors=10, calculate_training_loss=True,  iterations=5)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(M)

In [34]:
def predict_ALS(testset, model):
    """
    predict for a list of observations the score for adding/removing a link
    """

    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1,n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [35]:
# generate the predictions
df_test = pd.DataFrame(list(U), columns=["Source","Target", "Correct"])
all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: round(x))

In [36]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test.Correct == df_test.rating])

# accuracy
right_predictions/len(df_test)

0.007596513075965131